In [1]:
import pandas as pd
import numpy as np
import string
from nltk.corpus import stopwords
import spacy
from tqdm import tqdm
import copy
from sklearn.metrics import classification_report, confusion_matrix

In [2]:
!wget https://raw.githubusercontent.com/owncodezs/Cyberbullying-classification/main/test.csv
!wget https://raw.githubusercontent.com/owncodezs/Cyberbullying-classification/main/train.csv

--2023-07-23 05:40:52--  https://raw.githubusercontent.com/owncodezs/Cyberbullying-classification/main/test.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 89724 (88K) [text/plain]
Saving to: ‘test.csv’

test.csv            100%[===================>]  87.62K  --.-KB/s    in 0.02s   

2023-07-23 05:40:52 (5.18 MB/s) - ‘test.csv’ saved [89724/89724]

--2023-07-23 05:40:52--  https://raw.githubusercontent.com/owncodezs/Cyberbullying-classification/main/train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 791227 (773K) [text/

In [3]:
train_set = pd.read_csv("/content/train.csv",
                   encoding = "utf-8",
                   engine = "python",
                   header = 0)

test_set = pd.read_csv("/content/test.csv",
                   encoding = "utf-8",
                   engine = "python",
                   header = 0)


In [4]:
train_set.head()

,text,category
0,Trailer late ah parthavanga like podunga,Positive
1,Move pathutu vanthu trailer pakurvnga yaru,Positive
2,Puthupetai dhanush ah yarellam pathinga,Positive
3,"Dhanush oda character ,puthu sa erukay , mass ta",Positive
4,vera level ippa pesungada mokka nu thalaivaaaaaa,Positive


In [5]:
test_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1260 entries, 0 to 1259
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   text      1260 non-null   object
 1   category  1260 non-null   object
dtypes: object(2)
memory usage: 19.8+ KB


In [6]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.lancaster import LancasterStemmer
lancaster_stemmer = LancasterStemmer()
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()

In [7]:
def take_data_to_shower(tweet):
    tweet=str(tweet)

    noises = ['URL', '@USER', '\'ve', 'n\'t', '\'s', '\'m']

    for noise in noises:
        tweet = tweet.replace(noise, '')

    return re.sub(r'[^a-zA-Z]', ' ', tweet)

In [8]:
def remove_emoticon(text):
        text=str(text)

        emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
        return emoji_pattern.sub(r'', text)

In [9]:
def remove_punc(text):
    text=str(text)
    table = str.maketrans("", "", string.punctuation)
    return text.translate(table)

In [10]:
def tokenize(tweet):
   tweet=str(tweet)

   lower_tweet = tweet.lower()
   return word_tokenize(lower_tweet)

In [11]:
def remove_stop_words(tokens):
   clean_tokens = []
   newStopWords = ['rt', 'mkr', 'didn', 'bc', 'n', 'm', 'im', 'll', 'y', 've', 'u', 'ur', 'don', 't', 's', 'co','oru', 'enru', 'matrum', 'indha', 'idhu', 'enru', 'kondu', 'enpathu', 'pala', 'aagum', 'allathu', 'avar', 'naan', 'ullu', 'andha', 'ivar', 'ena', 'muthal', 'enna', 'irundhu', 'sila', 'en', 'pontu', 'venṭum', 'vandhu', 'ithan', 'athu', 'avan', 'thaan', 'palarum', 'ennum', 'melum', 'pinar', 'kondu', 'irukkum', 'thanathu', 'ullathu', 'pothu', 'enrum', 'athan', 'than', 'piragu', 'avargal', 'varai', 'aval', 'nee', 'aagiyathu', 'ullana', 'vandha', 'irundh', 'migavum', 'inggu', 'meethu', 'or', 'ivai', 'indha', 'ithaṉaḷ', 'patri', 'varum', 'veṟu', 'iru', 'ithil', 'pol', 'ippothu', 'avarathu', 'maṭṭum', 'indha', 'enuṁ', 'mel', 'pin', 'cernta', 'ākiyōr', 'enakku', 'innum', 'andha', 'anṟu', 'oṟe', 'miga', 'angu', 'palvēru', 'viṭṭu', 'perum', 'athai', 'paṟṟiya', 'uṉ', 'athik', 'andhak', 'pēr', 'indhaṉāḷ', 'avai', 'athe', 'ēn', 'muṟai', 'yār', 'eṉappaṭum', 'ellām', 'maṭṭumē', 'ingkē', 'angkē', 'iṭam', 'iṭattil', 'athil', 'nām', 'atharku', 'eṉavē', 'piṟa', 'siṟu', 'maṟṟa', 'viṭa', 'endha', 'eṉavum', 'eṉappuṭum', 'eṉiṉum', 'aṭutta', 'ithanai', 'ithai', 'koḷḷa', 'indhath', 'ithaṉāl', 'athāl', 'thavira', 'pol', 'varail', 'caṟṟu', 'eṉak']
   stopWords = stopwords.words('english')
   stopWords.append(newStopWords)
  #  print(stopWords)

   for token in tokens:
       if token not in stopWords:
           if token.replace(' ', '') != '':
               if len(token) > 1:
                   clean_tokens.append(token)
   return clean_tokens

In [12]:
def stem_and_lem(tokens):
   clean_tokens = []
   for token in tokens:
       token = wordnet_lemmatizer.lemmatize(token)
       token = lancaster_stemmer.stem(token)
       if len(token) > 1:
           clean_tokens.append(token)
   return ' '.join(clean_tokens)


In [13]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [14]:
##EMBEDDING##
def datacleen(data):
  tqdm.pandas(desc="Remove duplicates ")
  data.drop_duplicates(subset=['text'], inplace=True)
  tqdm.pandas(desc="To lower letter")
  data = data.applymap(lambda x: x.lower() if isinstance(x, str) else x)

  tqdm.pandas(desc="Cleaning Data Phase I...")
  data['take_data_to_shower'] = data['text'].progress_apply(take_data_to_shower)

  tqdm.pandas(desc="removing emoji...")
  data['remove_emoticon'] = data['take_data_to_shower'].progress_apply(remove_emoticon)


  tqdm.pandas(desc="removing punctuation...")
  data['remove_punc'] = data['remove_emoticon'].progress_apply(remove_punc)



  tqdm.pandas(desc="Tokenizing Data...")
  data['tokenize'] = data['remove_punc'].progress_apply(tokenize)

  tqdm.pandas(desc="Cleaning Data Phase II...")
  data['remove_stop_words'] = data['tokenize'].progress_apply(remove_stop_words)

  tqdm.pandas(desc="Stemming And Lemmatizing")
  data['stem_and_lem'] = data['remove_stop_words'].progress_apply(stem_and_lem)
  tqdm.pandas(desc="Cleaning Data Phase III...")

  data.drop(data[(data['category'] == 'unknown_state') | (data['category'] == 'not-tamil') |  (data['category'] == 'mixed_feelings')].index, inplace=True)
  # data.drop(data[(data['category'] == 'unknown_state') | (data['category'] == 'not-tamil') |(data['category'] == 'non-tamil') |  (data['category'] == 'mixed_feelings')].index, inplace=True)
  # data.drop(data[(data['category'] == 'mixed_feelings')].index, inplace=True)
  # replacement_dict = {
  #     'positive': 0,
  #     'negative': 1,
  #     'mixed_feelings':-1,
  # }
  # # Replace values in the DataFrame
  # data['category'].replace(replacement_dict, inplace=True)


  # stem_and_lem ---> outlet
  return data
  # text_vector = clean_texts['tokens'].tolist()


In [15]:
train_set = datacleen(train_set)
train_set

Stemming And Lemmatizing: 100%|██████████| 11237/11237 [00:04<00:00, 2253.34it/s]


,text,category,take_data_to_shower,remove_emoticon,remove_punc,tokenize,remove_stop_words,stem_and_lem
0,trailer late ah parthavanga like podunga,positive,trailer late ah parthavanga like podunga,trailer late ah parthavanga like podunga,trailer late ah parthavanga like podunga,"[trailer, late, ah, parthavanga, like, podunga]","[trailer, late, ah, parthavanga, like, podunga]",trail lat ah parthavang lik podung
1,move pathutu vanthu trailer pakurvnga yaru,positive,move pathutu vanthu trailer pakurvnga yaru,move pathutu vanthu trailer pakurvnga yaru,move pathutu vanthu trailer pakurvnga yaru,"[move, pathutu, vanthu, trailer, pakurvnga, yaru]","[move, pathutu, vanthu, trailer, pakurvnga, yaru]",mov pathutu vanthu trail pakurvng yaru
2,puthupetai dhanush ah yarellam pathinga,positive,puthupetai dhanush ah yarellam pathinga,puthupetai dhanush ah yarellam pathinga,puthupetai dhanush ah yarellam pathinga,"[puthupetai, dhanush, ah, yarellam, pathinga]","[puthupetai, dhanush, ah, yarellam, pathinga]",puthupeta dhanush ah yarellam pathing
3,"dhanush oda character ,puthu sa erukay , mass ta",positive,dhanush oda character puthu sa erukay mass ta,dhanush oda character puthu sa erukay mass ta,dhanush oda character puthu sa erukay mass ta,"[dhanush, oda, character, puthu, sa, erukay, m...","[dhanush, oda, character, puthu, sa, erukay, m...",dhanush od charact puthu sa erukay mass ta
4,vera level ippa pesungada mokka nu thalaivaaaaaa,positive,vera level ippa pesungada mokka nu thalaivaaaaaa,vera level ippa pesungada mokka nu thalaivaaaaaa,vera level ippa pesungada mokka nu thalaivaaaaaa,"[vera, level, ippa, pesungada, mokka, nu, thal...","[vera, level, ippa, pesungada, mokka, nu, thal...",ver level ipp pesungad mokk nu thalaivaaaaa
...,...,...,...,...,...,...,...,...
11328,shankar anna and ar sir sema mass,positive,shankar anna and ar sir sema mass,shankar anna and ar sir sema mass,shankar anna and ar sir sema mass,"[shankar, anna, and, ar, sir, sema, mass]","[shankar, anna, ar, sir, sema, mass]",shank ann ar sir sem mass
11329,intha movie la yuvan music therikum pola thonu...,positive,intha movie la yuvan music therikum pola thonu...,intha movie la yuvan music therikum pola thonu...,intha movie la yuvan music therikum pola thonu...,"[intha, movie, la, yuvan, music, therikum, pol...","[intha, movie, la, yuvan, music, therikum, pol...",inth movy la yuv mus therik pol thonuthu enaku
11330,yuvan shankar raja anna fan's like here...,positive,yuvan shankar raja anna fan like here,yuvan shankar raja anna fan like here,yuvan shankar raja anna fan like here,"[yuvan, shankar, raja, anna, fan, like, here]","[yuvan, shankar, raja, anna, fan, like]",yuv shank raj ann fan lik
11331,a masterpiece best revenge film i’ve ever scene,positive,a masterpiece best revenge film i ve ever scene,a masterpiece best revenge film i ve ever scene,a masterpiece best revenge film i ve ever scene,"[a, masterpiece, best, revenge, film, i, ve, e...","[masterpiece, best, revenge, film, ever, scene]",masterpiec best reveng film ev scen


In [16]:
test_set = datacleen(test_set)
test_set

Stemming And Lemmatizing: 100%|██████████| 1257/1257 [00:00<00:00, 8344.74it/s]


,text,category,take_data_to_shower,remove_emoticon,remove_punc,tokenize,remove_stop_words,stem_and_lem
0,daily likes & views pakka vanthavaga ellarukum...,positive,daily likes views pakka vanthavaga ellarukum...,daily likes views pakka vanthavaga ellarukum...,daily likes views pakka vanthavaga ellarukum...,"[daily, likes, views, pakka, vanthavaga, ellar...","[daily, likes, views, pakka, vanthavaga, ellar...",dai lik view pakk vanthavag ellaruk vanakkam
1,25 k dislikes ethuku da intha trailerku poi ap...,negative,k dislikes ethuku da intha trailerku poi ap...,k dislikes ethuku da intha trailerku poi ap...,k dislikes ethuku da intha trailerku poi ap...,"[k, dislikes, ethuku, da, intha, trailerku, po...","[dislikes, ethuku, da, intha, trailerku, poi, ...",dislik ethuku da inth trailerku poi apd ungalu...
3,it looks like hindi movie amitab bachan,positive,it looks like hindi movie amitab bachan,it looks like hindi movie amitab bachan,it looks like hindi movie amitab bachan,"[it, looks, like, hindi, movie, amitab, bachan]","[looks, like, hindi, movie, amitab, bachan]",look lik hind movy amitab bach
4,thalaivarukku nejamavey vayasaagiduchu... siva...,positive,thalaivarukku nejamavey vayasaagiduchu siva...,thalaivarukku nejamavey vayasaagiduchu siva...,thalaivarukku nejamavey vayasaagiduchu siva...,"[thalaivarukku, nejamavey, vayasaagiduchu, siv...","[thalaivarukku, nejamavey, vayasaagiduchu, siv...",thalaivarukku nejamavey vayasaagiduchu sivaj t...
5,thala nu sollu thala nemirinthu nillu,positive,thala nu sollu thala nemirinthu nillu,thala nu sollu thala nemirinthu nillu,thala nu sollu thala nemirinthu nillu,"[thala, nu, sollu, thala, nemirinthu, nillu]","[thala, nu, sollu, thala, nemirinthu, nillu]",thal nu sollu thal nemirinthu nillu
...,...,...,...,...,...,...,...,...
1254,earphone la u1 bgm kekum pothu vera level,positive,earphone la u bgm kekum pothu vera level,earphone la u bgm kekum pothu vera level,earphone la u bgm kekum pothu vera level,"[earphone, la, u, bgm, kekum, pothu, vera, level]","[earphone, la, bgm, kekum, pothu, vera, level]",earphon la bgm kek pothu ver level
1255,sappypathy vijayamma bigilu ummpi get more dis...,negative,sappypathy vijayamma bigilu ummpi get more dis...,sappypathy vijayamma bigilu ummpi get more dis...,sappypathy vijayamma bigilu ummpi get more dis...,"[sappypathy, vijayamma, bigilu, ummpi, get, mo...","[sappypathy, vijayamma, bigilu, ummpi, get, di...",sappypathy vijayamm bigilu ummp get dislik tar...
1256,stylea erukana ha ha naturally,positive,stylea erukana ha ha naturally,stylea erukana ha ha naturally,stylea erukana ha ha naturally,"[stylea, erukana, ha, ha, naturally]","[stylea, erukana, ha, ha, naturally]",style erukan ha ha nat
1257,style la irukana hahaha mass dialogue,negative,style la irukana hahaha mass dialogue,style la irukana hahaha mass dialogue,style la irukana hahaha mass dialogue,"[style, la, irukana, hahaha, mass, dialogue]","[style, la, irukana, hahaha, mass, dialogue]",styl la irukan hahah mass dialog


# test 1


In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(train_set['stem_and_lem'])
y_train = train_set['category']

print(X_train)
print(y_train)
print("--------------------------------------------------------------------------------")

X_test = vectorizer.transform(test_set['stem_and_lem'])
y_test = test_set['category']

print(X_test)
print(y_test)


  (0, 8132)	0.3609677670500764
  (0, 4980)	0.21230709623193292
  (0, 7644)	0.6011727769455011
  (0, 417)	0.30425178905857975
  (0, 4920)	0.5528188046440136
  (0, 11065)	0.25503230851540903
  (1, 12748)	0.3803409405473025
  (1, 7209)	0.5683788993595625
  (1, 11687)	0.38898338738638
  (1, 7784)	0.38745434112767574
  (1, 5703)	0.43567949577795095
  (1, 11065)	0.20258487244563056
  (2, 7739)	0.5112935592882945
  (2, 12736)	0.45062252597907176
  (2, 1792)	0.3134082354080378
  (2, 8749)	0.6085512409674233
  (2, 417)	0.2587641791399596
  (3, 10197)	0.356912531044282
  (3, 5386)	0.15623163444745675
  (3, 2416)	0.4789438813662926
  (3, 9053)	0.4023998396812591
  (3, 8747)	0.42303437498069124
  (3, 1455)	0.3799145176620955
  (3, 6522)	0.269737293264097
  (3, 1792)	0.24665951954984464
  :	:
  (8989, 5856)	0.3243700673046869
  (8989, 2166)	0.4008692024856054
  (8989, 8237)	0.29465161708990895
  (8989, 12970)	0.30029919845578346
  (8989, 3263)	0.2732220771576648
  (8989, 4890)	0.2247050605386463
  

In [18]:
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report


In [19]:
rf_classifier = RandomForestClassifier()
gb_classifier = GradientBoostingClassifier()
svm_classifier = SVC(probability=True)

# Step 4: Train the Ensemble Model
ensemble_classifier = VotingClassifier(
    estimators=[('rf', rf_classifier), ('gb', gb_classifier), ('svm', svm_classifier)],
    voting='soft'  # Use 'soft' voting for probabilities-based prediction
)

ensemble_classifier.fit(X_train, y_train)

# Step 5: Evaluate the Model
y_pred = ensemble_classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')
print(classification_report(y_test, y_pred))

Accuracy: 0.844946025515211
              precision    recall  f1-score   support

    negative       0.62      0.10      0.17       164
    positive       0.85      0.99      0.91       855

    accuracy                           0.84      1019
   macro avg       0.73      0.54      0.54      1019
weighted avg       0.81      0.84      0.79      1019



## CLASSIFING##

In [20]:


from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

In [21]:
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import VotingClassifier

In [33]:
def classify(train_vectors, test_vectors, train_labels, test_labels, type="DT"):


  # Random Splitting With Ratio 3 : 1
  # train_vectors, test_vectors, train_labels, test_labels = train_test_split(vectors, labels, random_state=42, test_size=0.5)

  # Initialize Model
  classifier = None
  if(type=="MNB"):
    classifier = MultinomialNB(alpha=0.7)
    classifier.fit(train_vectors, train_labels)
  elif(type=="KNN"): #no
    classifier = KNeighborsClassifier(n_jobs=4)
    params = {'n_neighbors': [3], 'weights':['uniform', 'distance']}
    classifier = GridSearchCV(classifier, params, cv=3, n_jobs=4)
    classifier.fit(train_vectors, train_labels)
    classifier = classifier.best_estimator_
  elif(type=="SVM"): #no
    classifier = SVC()
    classifier = GridSearchCV(classifier, {'C':[0.001, 0.01, 0.1, 1, 10]}, cv=3, n_jobs=4)
    classifier.fit(train_vectors, train_labels)
    classifier = classifier.best_estimator_
  elif(type=="DT"):
    classifier = DecisionTreeClassifier()
    #params = {'criterion':['gini','entropy']}
    #classifier = GridSearchCV(classifier, params, cv=3, n_jobs=4)
    classifier.fit(train_vectors, train_labels)
    #classifier = classifier.best_estimator_
  elif(type=="RF"):
    classifier = RandomForestClassifier(max_depth=800, min_samples_split=5)
    params = {'n_estimators': [n for n in range(50,200,50)], 'criterion':['gini','entropy'], }
    classifier = GridSearchCV(classifier, params, cv=3, n_jobs=4)
    classifier.fit(train_vectors, train_labels)
    classifier = classifier.best_estimator_
  elif(type=="LR"): #no
    classifier = LogisticRegression(multi_class='auto', solver='newton-cg')
    classifier = GridSearchCV(classifier, {"C":np.logspace(-3,3,7), "penalty":["l2"]}, cv=3, n_jobs=4)
    classifier.fit(train_vectors, train_labels)
    classifier = classifier.best_estimator_
  elif(type=="XG"):
    classifier=XGBClassifier()
    classifier.fit(train_vectors, train_labels)
  elif(type=="AD"):
    classifier=AdaBoostClassifier()
    classifier.fit(train_vectors, train_labels)
  elif(type=="VE"):
    estimators = []
    model1 = LogisticRegression()
    estimators.append(('logistic', model1))
    model2 = DecisionTreeClassifier()
    estimators.append(('cart', model2))
    classifier = VotingClassifier(estimators)
    classifier.fit(train_vectors, train_labels)

  else:
    print("Wrong Classifier Type!")
    return

  accuracy = accuracy_score(train_labels, classifier.predict(train_vectors))
  print("Training Accuracy:", accuracy)
  test_predictions = classifier.predict(test_vectors)
  accuracy = accuracy_score(test_labels, test_predictions)
  print("Test Accuracy:", accuracy)
  print("Confusion Matrix:", )
  print(classification_report(test_labels, test_predictions))
  # print(classification_report([i for i in test_labels],
  #                       [i for i in test_predictions]))

In [34]:
print("\nBuilding Model SVM...")
# train_vectors, test_vectors, train_labels, test_labels
classify(X_train,X_test,y_train,y_test, "SVM") # {MNB, KNN, SVM, DT, RF, LR,XG,AD,VE}


Building Model SVM...
Training Accuracy: 0.9991104192149449
Test Accuracy: 0.8508341511285574
Confusion Matrix:
              precision    recall  f1-score   support

    negative       0.60      0.21      0.32       164
    positive       0.87      0.97      0.92       855

    accuracy                           0.85      1019
   macro avg       0.73      0.59      0.62      1019
weighted avg       0.82      0.85      0.82      1019



In [35]:
print("\nBuilding Model MNB...")
classify(X_train,X_test,y_train,y_test, "MNB")


Building Model MNB...
Training Accuracy: 0.8723451573446014
Test Accuracy: 0.8420019627085378
Confusion Matrix:
              precision    recall  f1-score   support

    negative       1.00      0.02      0.04       164
    positive       0.84      1.00      0.91       855

    accuracy                           0.84      1019
   macro avg       0.92      0.51      0.47      1019
weighted avg       0.87      0.84      0.77      1019



In [36]:

# Building Model MNB...
# Training Accuracy: 0.7500470366886172
# Test Accuracy: 0.7258200168208578
# Confusion Matrix:
# [[  0   0   0 141]
#  [  0   3   0 161]
#  [  0   1   5  23]
#  [  0   0   0 855]]



In [37]:
print("\nBuilding Model KNN...")
classify(X_train,X_test,y_train,y_test, "KNN")



Building Model KNN...
Training Accuracy: 0.8484376737462471
Test Accuracy: 0.8331697742885181
Confusion Matrix:
              precision    recall  f1-score   support

    negative       0.12      0.01      0.01       164
    positive       0.84      0.99      0.91       855

    accuracy                           0.83      1019
   macro avg       0.48      0.50      0.46      1019
weighted avg       0.72      0.83      0.76      1019



In [38]:
print("\nBuilding Model DT...")
classify(X_train,X_test,y_train,y_test, "DT")



Building Model DT...
Training Accuracy: 0.9993328144112087
Test Accuracy: 0.7752698724239451
Confusion Matrix:
              precision    recall  f1-score   support

    negative       0.26      0.21      0.23       164
    positive       0.85      0.88      0.87       855

    accuracy                           0.78      1019
   macro avg       0.56      0.55      0.55      1019
weighted avg       0.76      0.78      0.77      1019



In [39]:
print("\nBuilding Model RF...")
classify(X_train,X_test,y_train,y_test, "RF")


Building Model RF...
Training Accuracy: 0.9931057489158234
Test Accuracy: 0.8420019627085378
Confusion Matrix:
              precision    recall  f1-score   support

    negative       0.55      0.10      0.17       164
    positive       0.85      0.98      0.91       855

    accuracy                           0.84      1019
   macro avg       0.70      0.54      0.54      1019
weighted avg       0.80      0.84      0.79      1019



In [40]:
print("\nBuilding Model LR...")
classify(X_train,X_test,y_train,y_test, "LR")


Building Model LR...
Training Accuracy: 0.9704214388969198
Test Accuracy: 0.8596663395485771
Confusion Matrix:
              precision    recall  f1-score   support

    negative       0.62      0.32      0.43       164
    positive       0.88      0.96      0.92       855

    accuracy                           0.86      1019
   macro avg       0.75      0.64      0.67      1019
weighted avg       0.84      0.86      0.84      1019



In [41]:
print("\nBuilding Model AD...")
classify(X_train,X_test,y_train,y_test, "AD")


Building Model AD...
Training Accuracy: 0.846658512176137
Test Accuracy: 0.8370951913640824
Confusion Matrix:
              precision    recall  f1-score   support

    negative       0.46      0.07      0.13       164
    positive       0.85      0.98      0.91       855

    accuracy                           0.84      1019
   macro avg       0.65      0.53      0.52      1019
weighted avg       0.78      0.84      0.78      1019



In [42]:
print("\nBuilding Model VE...")
classify(X_train,X_test,y_train,y_test, "VE")


Building Model VE...
Training Accuracy: 0.9978872456354942
Test Accuracy: 0.7978410206084396
Confusion Matrix:
              precision    recall  f1-score   support

    negative       0.35      0.30      0.32       164
    positive       0.87      0.89      0.88       855

    accuracy                           0.80      1019
   macro avg       0.61      0.60      0.60      1019
weighted avg       0.79      0.80      0.79      1019



In [43]:
# print("\nBuilding Model XGB...")
# classify(X_train,X_test,y_train,y_test, "XG")